# Crawling

## Mydramalist

In [1]:
import time
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
options =  webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
chrome_driver_path = '/opt/homebrew/Caskroom/chromedriver/100.0.4896.60/chromedriver'

In [ ]:
def scroll_down(second):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(second)
    new = driver.execute_script("return document.documentElement.scrollHeight")
    
    return new

In [ ]:
driver = webdriver.Chrome(chrome_driver_path, options=options)
for k,v in urls.items():
    reviews_dict = {
    'title' : [],
    'reviews' : []
    }
    driver.get(v)
    time.sleep(4)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    time.sleep(2)
    while True:
        new_height = scroll_down(2)
        if new_height == last_height:
            try:
                button = driver.find_element_by_css_selector('#cmtsapp > div.box-footer > button')
                driver.execute_script("arguments[0].click();", button)
                time.sleep(1)
            except:
                break
        last_height = new_height
    
    buttons = driver.find_elements_by_class_name('reveal')
    for button in tqdm(buttons):
        driver.execute_script("arguments[0].click();", button)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    reviews = soup.find_all("div", {"class": "post-message"})
    
    for review in reviews:
        reviews_dict['title'].append(k)
        reviews_dict['reviews'].append(review.p.text)
    
    df = pd.DataFrame(reviews_dict)
    csv = df.to_csv(f"/Users/suchan/study/파이널 프로젝트/{k}_mydramalist.csv")
    csv = pd.read_csv(f"/Users/suchan/study/파이널 프로젝트/{k}_mydramalist.csv")
    csv.drop(['Unnamed: 0'], axis=1, inplace=True)
    
    print(f"{k} 끝!!") 

## Letterboxd

In [ ]:
drama_dict = {'스물다섯 스물하나' : 'https://letterboxd.com/film/twenty-five-twenty-one/'}

In [ ]:
reviews_dic = {'title' : [], 'reviews' : []}

In [ ]:
driver = webdriver.Chrome(chrome_driver_path, options=options)
for k,v in drama_dict.items():
    url = v
    page_break = False
    for i in range(1,100):
        driver.get(url+f'reviews/by/activity/page/{i}/')
        time.sleep(3)
        
        # 타이틀 갖고오기
        title = driver.find_element_by_xpath('/html/body/div[3]/div/div/section/header/div/h1/a').text
        
        # more랑 handle 버튼 누르기 없으면 그대로 진행
        try:
            buttons = driver.find_elements_by_class_name('js-reveal')
            for button in buttons:
                try:
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(2)
                except:
                    continue
        except:
            continue
        
        # 리뷰 갖고오기
        for j in range(1, 13):
            try:
                review = driver.find_element_by_xpath(f'/html/body/div[3]/div/div/section/section/ul/li[{j}]/div/div[2]').text
                reviews_dic['title'].append(title)
                reviews_dic['reviews'].append(review)
            except:
                # 리뷰값이 더 없으면 리뷰 for문과 페이지 for문 중단
                page_break = True
                break
        if (page_break == True):
            break
                
    time.sleep(2)

print(f'{k} 끝!!')

## IMDB

In [ ]:
driver = webdriver.Chrome('/opt/homebrew/Caskroom/chromedriver/99.0.4844.51/chromedriver')
driver.set_page_load_timeout("20")

# 작품 구분없이 장르별 드라마 10개의 리뷰 가져오기
for i in range(3):
    for j in range(10):
        url = WHOLE_URL[i + 4][j]
        time.sleep(5)
        driver.get(url)
        
        # 작품 제목
#         title = driver.find_element_by_css_selector('#main > section > div.subpage_title_block > div > div > h3 > a').text
#         print("title", title)
        
        # 댓글 수
        review_counts = driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[3]/div[1]/section/div[2]/div[1]/div/span').text
        review_counts = int(review_counts.split(' ')[0].replace(',', ''))
        
        # Load More 끝날 때까지 반복
        for k in range(review_counts):    
            try:
                driver.find_element_by_class_name("ipl-load-more__button").send_keys(Keys.ENTER)
                time.sleep(3)
            except Exception as ex:
                print('더이상 "Load More" 없음', ex)
                break

        # 댓글 수집
        print(" 댓글 수집 ")
        try:
            time.sleep(0.5)
            soup= BeautifulSoup(driver.page_source)
            time.sleep(1)

            comment=[]
            x= soup.find_all("div", class_="content")

            for h in x:
                try:
                    tmp= h.find_all("div", class_="text show-more__control")[0].text
                    review_dic['genre'].append(genre[i + 4])
                    review_dic['review'].append(tmp)
                    
                except Exception as ex:
                    tmp= h.find_all("div", class_="text show-more__control clickable")[0].text
                    review_dic['genre'].append(genre[i + 4])
                    review_dic['review'].append(tmp)

            time.sleep(1)
        except Exception as ex:
            re=['nothing']
            print("댓글없음")

        # 한줄평
        print("한줄평 수집 ")
        try:
            oneline=[]
            x= soup.find_all("div", class_="lister-item-content")
            for h in x:
                ol= h.find_all("a", class_="title")[0].text
                review_dic['genre'].append(genre[i + 4])
                review_dic['review'].append(ol)
                
            time.sleep(0.5)

        except Exception as ex:
            oneline= 'nothing'
            print("")

        time.sleep(1)
        print(datetime.now())
        print("---수집 댓글, 한줄평 저장---")

## Concat

In [ ]:
titles = Mydramalist.title.unique().tolist()

In [ ]:
for title in titles:
    mydramalist = pd.DataFrame(Mydramalist[Mydramalist.title == title].review.tolist())
    letterboxd = pd.DataFrame(Letterboxd[Letterboxd.title == title].review.tolist())
    imdb = pd.DataFrame(IMDB[IMDB.title == title].review.tolist())
    df = pd.concat([mydramalist,letterboxd,imdb], axis=0)
    df.to_csv(f"{title}_reviews.csv",index=False)

# EDA

In [ ]:
sentences = []

for i in df_watcha['reviews']:
    text = re.sub('[^a-zA-Z\']',' ',i).strip()
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
    # text = re.sub(' +', ' ', text)
    text = ' '.join(text.split())
    sentences.append(text)

In [ ]:
eng_frame = pd.DataFrame(sentences, columns = ['reviews'])

In [ ]:
samples = pd.Series(eng_frame['reviews'])

In [ ]:
df_length = samples.apply(len)
eng_frame['lens'] = df_length

In [ ]:
print('리뷰 길이 최대값 : {}'.format(np.max(eng_frame['lens'])))
print('리뷰 길이 최소값 : {}'.format(np.min(eng_frame['lens'])))
print('리뷰 길이 평균값 : {:.2f}'.format(np.mean(eng_frame['lens'])))
print('리뷰 길이 표준편차 : {:.2f}'.format(np.std(eng_frame['lens'])))
print('리뷰 길이 중간값 : {}'.format(np.median(eng_frame['lens'])))
print('리뷰 길이 제1사분위 : {}'.format(np.percentile(eng_frame['lens'], 25)))
print('리뷰 길이 제3사분위 : {}'.format(np.percentile(eng_frame['lens'], 75)))

In [ ]:
length = np.percentile(df_length, 25)

In [ ]:
eng_reviews= []
for index, i in enumerate(df_length):
    if i > length and i < 13000:
        eng_reviews.append(eng_frame.reviews[index])
    else:
        eng_reviews.append('빈값')

In [ ]:
df_eng = pd.DataFrame(eng_reviews, columns = ['reviews'])
df_eng['title'] = titles

In [ ]:
df_eng.drop(df_eng[df_eng['reviews'] == '빈값'].index)

In [ ]:
final_eng['title'].unique()

# Pre-processing

In [ ]:
from langdetect import detect
import re
import pandas as pd
import numpy as np
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize

In [ ]:
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang

In [ ]:
df['Lang'] = df['reviews'].apply(det)
en_reviews = (df['Lang'] == 'en')
df_en_reviews = df[en_reviews]['reviews']
detect_list = df_en_reviews.to_list()

In [ ]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, a):
        a = ''.join(a)
        word_tokens = self.tagger(a)
        
        words = []

        for i in word_tokens:
            text = re.sub('[^a-zA-Z0-9\']','',i).strip()
            text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`…》]','', text)
            if(text != ''):
                words.append(text)
        
        tag_words = nltk.pos_tag(words)
        # pos_words = [word for word in tag_words if word[1][0] in {'V','N','J','R'}]
        pos_words = []
        for word in tag_words:
            if word[1] in ['NNP', 'NN']:  #'VB', 'VBP', 'JJ'
                pos_words.append(word)
        temp_list = []
        for token, pos_tag in pos_words:
            tag = get_wordnet_pos(pos_tag)
            if tag != None:
                temp_list.append((token, get_wordnet_pos(pos_tag)))
        lemma = WordNetLemmatizer()
        token_final = [lemma.lemmatize(token, pos=tag) for token, tag in temp_list]
        long_words = [i for i in token_final if len(i) > 2]
        results = [w for w in long_words if w not in stop_words]
        return results

In [ ]:
tokens = []
custom_tokenizer = CustomTokenizer(text_to_word_sequence)
for sentences in detect_list:
    tokens.append(custom_tokenizer(sentences))

# Korean Category

## Review Classification

In [ ]:
df_lower = df['reviews'].str.lower()

In [ ]:
find_word_list = set(['korean idol', ' korean star', 'running man', 'nct', 'blackpink', 'bts', 'exo', 'exo l',
                      'iu', 'bona', 'wjsn', 'kdrama look', 'hanbok', 'korean clothing', 'sseugaechima', 'outfits', 'airport outfit',
                      'chimaek', 'chimac', 'mukbang', 'kimchi', 'kimchee', 'kimchijigae', 'ramen', 'hansik', 'haejangguk',
                      'korean cook', 'korean bbq', 'korean barbeque', 'soju', 'korean drinks', 'korean noodle', 'ksool', 'sool',
                      'dalgona', 'makgeolli', 'cold noodles', 'kfood', 'tteokbokki', 'topokki', 'ramyeon', 'samgyeopsal',
                      'bossam', 'ost', 'by taeyeon', 'kpop', 'kbeauty', 'korean beauty', 'oppa', 'unni', 'unnie', 'noona',
                      'daebak', 'heol', 'jjang', 'aegyo', 'simkung', 'kkuljaem', 'nojaem', 'hwaiting', 'menbung', 'selka',
                      'jjeoreo', 'eotteoke', 'sesangae', 'nunting', 'bulgeum', 'matjeom', 'honsool', 'honsul', 'honbab', 'jonmat', 'kol',
                      'ssum', 'sseom', 'mossol', 'tmi', 'green light', 'gapjil', 'kkondae', 'deokhu', 'deokjil', 'finger heart', 'maknae',
                      'mujigae', 'chaebol', 'gajima', 'makjang', 'gganbu', 'fighting', 'vincenzo', 'moon embracing the sun', 'hallyu', 'korean wave',
                      'hide and seek', 'chingu', 'manhwa', 'slowburn', 'slow burn', 'kzombie', 'south korea', 's korea', 'south koreas', 'parasite', 'gwangju uprising',
                      'north korea', 'sageuk', 'sageuks', 'marbles', 'seoul', 'busan', 'kdrama', 'kbs', 'sbs', 'mbc', 'jtbc', 'tvn', 'korean', 'koreans',
                      'chosun', 'joseon', 'manga', 'ganbu', 'incheon', 'daejeon', 'daegu', 'gangnam', 'jeonju', 'jeollabuk do', 'gyeonggi', 'dongseong',
                      'korea', 'hanguk', 'baeksang', 'ddakji', 'joseon', 'goryeo', 'gojong', ' shinmiyangyo', 'kdramas', 'cosmetic surgery', 'plastic surgery',
                      'jjampong', 'jajangmyeon', 'nurungji', 'gukbap', 'kimbap', 'bibimbap', 'sikhye', 'gimbap', 'lee sun hee', 'osts', 'lee sun hees', 'taeyeon',
                      'ben', 'davichi', 'punch', 'sheet mask', 'zombie makeup', 'lipstick', 'netizen', 'ahjumma', 'ajeossi', 'skinship', 'morning call',
                      'gimpo', 'gimhae', 'guro', 'gumi', 'gwacheon', 'sungkyunkwan', 'jeju', 'hwaseong', 'sungkyunkwan scandal', 'yi san', 'yi sans', 'sado',
                      'hwaseong killer', 'hwaseong serial killer', 'taekwondo', 'tug of war', 'ktweet', 'kwon il yong', 'taeyeons', 'tteok', 'kang ho sun',
                      'kwon ilyongs', 'itzy', 'enhypen', 'nuest', 'girls generation', 'clothing', 'outfit', 'fashion show', 'smart fashion', 'banchan',
                      'ailee', 'gaja', 'train to busan', 'dream high', 'producers', 'hanyang', 'jeon bong jun', 'donghak', 'raincoat killer', 'beanpole',
                      'uniform', 'lee san', 'track record', 'bigbang', 'minari', 'taegeuk', 'imjin', 'chuseok', 'nongshim', 'itaewon', 'hongdae',
                     'appa', 'umma', 'eomma', 'dongsaeng', 'dongseng', 'imo', 'sajang', 'saranghe', 'sunbae', 'seonbae', 'samchon', 'halmeoni', 'halabeoji',
                      'jinttobaegi', 'sseraegi', 'podaegi', 'sangsa', 'lee moon saes', 'lee moon sae', 'seunghwan', 'melomance', 'sondia', 'heize',
                      'hwasa', 'do bong soon', 'kim eun hee', 'goguryeo', 'silla', 'baekje', 'baekhyun', 'mamamoo', 'kim eunsook', 'kim eun sook',
                      'kim eun suk', 'kim eunsuk'])

In [ ]:
K_dict = {'K_words_list' : []}
for review in reviews:
    word_list = []
    for word in find_word_list:
        if re.search(f"[\W]{word}[\W]",review):
            word_list.append(word)
    if len(word_list) != 0:
        K_dict['K_words_list'].append(word_list)
    else:
        K_dict['K_words_list'].append(0)

In [ ]:
df_K_dict = pd.DataFrame(K_dict)

In [ ]:
df_ = df_K_dict.replace(0, np.NaN)

In [ ]:
df_korean_flavor = pd.concat([df_lower,df_], axis=1)
df_korean_flavor.dropna(inplace=True)

In [ ]:
k_food = [
    'chimaek', 'chimac', 'mukbang', 'kimchi', 'kimchee', 'kimchijigae', 'ramen', 'hansik', 'haejangguk', 'korean cook', 'korean bbq',
    'korean barbeque', 'soju', 'korean drinks', 'korean noodle', 'ksool', 'sool', 'makgeolli', 'cold noodles', 'kfood', 'tteokbokki',
    'topokki', 'ramyeon', 'samgyeopsal', 'jjampong', 'jajangmyeon', 'gimbap', 'nurungji', 'gukbap', 'kimbap', 'bibimbap', 'sikhye',
    'tteok', 'banchan', 'nongshim', 'bossam'
]

place = [
    'south korea', 's korea', 'south koreas', 'north korea', 'seoul', 'busan', 'incheon', 'daejeon', 'daegu', 'gangnam',
    'jeonju', 'jeollabuk do', 'gyeonggi', 'dongseong', 'korea', 'hanguk', 'gimpo', 'gimhae', 'guro', 'gumi', 'gwacheon',
    'sungkyunkwan', 'jeju', 'hwaseong', 'hanyang', 'itaewon', 'hongdae'
]

k_slang = [
    'oppa', 'unni', 'unnie', 'noona', 'daebak', 'heol', 'jjang', 'aegyo', 'simkung', 'kkuljaem', 'nojaem', 'hwaiting', 'menbung', 'selka',
    'jjeoreo', 'eotteoke', 'sesangae', 'nunting', 'bulgeum', 'matjeom', 'honsool', 'honsul', 'honbab', 'jonmat', 'kol', 'ssum', 'sseom',
    'mossol', 'green light', 'gapjil', 'kkondae', 'deokhu', 'deokjil', 'maknae', 'mujigae', 'chaebol', 'gajima', 'makjang', 'gganbu', 'ganbu',
    'netizen', 'ahjumma', 'ajeossi', 'skinship', 'morning call', 'gaja', 'appa', 'umma', 'eomma', 'dongsaeng', 'dongseng', 'imo', 'sajang', 'saranghe',
    'sunbae', 'seonbae', 'samchon', 'halmeoni', 'halabeoji', 'jinttobaegi', 'sseraegi', 'podaegi', 'sangsa', 'fighting'
]

k_fashion_beauty = [
    'kdrama look', 'hanbok', 'korean clothing', 'sseugaechima', 'beanpole', 'kbeauty', 'korean beauty', 'sheet mask', 'zombie makeup',
    'outfits', 'airport outfit', 'clothing', 'outfit', 'fashion show', 'smart fashion', 'uniform', 'cosmetic surgery', 'plastic surgery', 'lipstick'
]

k_pop = [
    'ost', 'lee sun hee', 'osts', 'lee sun hees', 'taeyeon', 'taeyeons', 'ben', 'davichi', 'ailee', 'nuest', 'track record',
    'lee moon saes', 'lee moon sae', 'seunghwan', 'melomance', 'sondia', 'heize', 'baekhyun', 'korean idol', 'korean star', 'nct',
    'blackpink', 'bts', 'd.o.', 'exo', 'exo l', 'iu', 'bona', 'wjsn', 'tvxq', 'itzy', 'enhypen', 'girls generation', 'bigbang', 'hwasa', 'mamamoo'
]

history_event = [
    'joseon', 'goryeo', 'gojong', 'shinmiyangyo', 'gwangju uprising',
    'yi san', 'yi sans', 'sado', 'kwon ilyongs', 'kwon il yong', 'kang ho sun', 'hwaseong killer',
    'hwaseong serial killer', 'jeon bong jun', 'donghak', 'raincoat killer', 'lee san', 'imjin', 'chosun', 'goguryeo', 'silla', 'baekje'
]

k_culture = [
    'vincenzo', 'moon embracing the sun', 'parasite', 'train to busan', 'dream high', 'producers', 'minari', 'running man',
    'sungkyunkwan scandal', 'do bong soon', 'hallyu', 'korean wave', 'hide and seek', 'chingu', 'manhwa', 'slowburn', 'slow burn',
    'kzombie', 'sageuk', 'sageuks', 'marbles', 'baeksang', 'ddakji', 'taekwondo', 'finger heart', 'dalgona', 'taegeuk', 'chuseok',
    'kim eun hee', 'kim eunsook', 'kim eun sook', 'kim eun suk', 'kim eunsuk', 'kdrama', 'kdramas', 'kbs', 'sbs', 'mbc', 'jtbc', 'tvn',
    'korean', 'koreans', 'manga', 'tug of war', 'k tweet'
]

In [ ]:
categories = [k_food, place, k_slang, k_fashion_beauty, k_pop, history_event,k_culture]

In [ ]:
numpy1 = np.zeros((27699,7))
category_df = pd.DataFrame(numpy1)

In [ ]:
df_korean_flavor.reset_index(inplace=True)
df_korean_flavor.drop(['index'],axis=1,inplace=True)

In [ ]:
for category in categories:
    for i in tqdm(range(len(df_korean_flavor))):
        length = len(df_korean_flavor.K_words_list[i])
        for j in range(length):
            if df_korean_flavor.K_words_list[i][j] in category:
                category_df.iloc[i][categories.index(category)] += 1

In [ ]:
category_df.columns= ['k_food', 'place', 'k_slang', 'k_fashion_beauty', 'k_pop', 'history_event', 'k_culture']

In [ ]:
df_korean = pd.concat([df_korean_flavor,category_df],axis=1)

In [ ]:
df_korean.to_csv("ENG_Korean.csv", index=False)

## Count Topic & Rank Topic

In [ ]:
korea_senti = pd.read_csv('/content/drive/MyDrive/0506_ENG_Korean_plus_Sentiment.csv')

In [ ]:
k_food = []
place = []
k_slang = []
k_fashion_beauty = []
k_pop = []
history_event = []
k_culture = []
titles = []
for title in korea_senti['title'].unique():
    senti = korea_senti[korea_senti['title'] == title]
    k_food.append(senti['k_food'].sum())
    place.append(senti['place'].sum())
    k_slang.append(senti['k_slang'].sum())
    k_fashion_beauty.append(senti['k_fashion_beauty'].sum())
    k_pop.append(senti['k_pop'].sum())
    history_event.append(senti['history_event'].sum())
    k_culture.append(senti['k_culture'].sum())
    titles.append(title)

In [ ]:
df = pd.DataFrame(list(zip(titles, k_food, place, k_slang, k_fashion_beauty, k_pop, history_event, k_culture, df.sum(axis=1))), columns = ['titles', 'k_food', 'place', 'k_slang', 'k_fashion_beauty', 'k_pop', 'history_event', 'k_culture', 'keywords_sum'])

In [ ]:
ratio_food = df['k_food']/df['keywords_sum']
ratio_place = df['place']/df['keywords_sum']
ratio_slang = df['k_slang']/df['keywords_sum']
ratio_fashion_beauty = df['k_fashion_beauty']/df['keywords_sum']
ratio_pop = df['k_pop']/df['keywords_sum']
ratio_history_event = df['history_event']/df['keywords_sum']
ratio_culture = df['k_culture']/df['keywords_sum']

In [ ]:
df['ratio_food'] = ratio_food
df['ratio_place'] = ratio_place
df['ratio_slang'] = ratio_slang
df['ratio_fashion_beauty'] = ratio_fashion_beauty
df['ratio_pop'] = ratio_pop
df['ratio_history_event'] = ratio_history_event
df['ratio_culture'] = ratio_culture

In [ ]:
rank_food = df['ratio_food'].rank(method='min', ascending=False)
rank_place = df['ratio_place'].rank(method='min', ascending=False)
rank_slang = df['ratio_slang'].rank(method='min', ascending=False)
rank_fashion_beauty = df['ratio_fashion_beauty'].rank(method='min', ascending=False)
rank_pop = df['ratio_pop'].rank(method='min', ascending=False)
rank_history_event = df['ratio_history_event'].rank(method='min', ascending=False)
rank_culture = df['ratio_culture'].rank(method='min', ascending=False)

In [ ]:
df['rank_food'] = rank_food
df['rank_place'] = rank_place
df['rank_slang'] = rank_slang
df['rank_fashion_beauty'] = rank_fashion_beauty
df['rank_pop'] = rank_pop
df['rank_history_event'] = rank_history_event
df['rank_culture'] = rank_culture

In [ ]:
eng_group = pd.read_csv('/content/drive/MyDrive/ENG_drama_group.csv', index_col=0)

In [ ]:
df_rank = eng_group[['titles', 'rank_food', 'rank_place', 'rank_slang', 'rank_fashion_beauty', 'rank_pop', 'rank_history_event', 'rank_culture']]

In [ ]:
copy_df.replace({'rank_food' : 43}, 100, inplace=True)
copy_df.replace({'rank_place' : 92}, 100, inplace=True)
copy_df.replace({'rank_slang' : 92}, 100, inplace=True)
copy_df.replace({'rank_fashion_beauty' : 68}, 100, inplace=True)
copy_df.replace({'rank_pop' : 91}, 100, inplace=True)
copy_df.replace({'rank_history_event' : 40}, 100, inplace=True)

In [ ]:
rank_copy = df_rank.copy()
rank_copyt = rank_copy.T
rank_list = rank_copy['titles'].to_list()

In [ ]:
rank_copyt.columns = rank_list
rank_copyt = rank_copyt.drop('titles', axis = 0)

## Drama Grouping

In [ ]:
under_20 = []
ind = rank_copyt['index']
for title in rank_list:
    temp = []
    index_list = np.argsort(rank_copyt[title])
    for index in tqdm(index_list[:2]):
        if (rank_copyt[title].iloc[index] <= 20) & (len(temp) <= 1):
            temp.append(ind[index])
            # rank_sort = np.argsort(rank_copyt[title])
            # temp.append(rank_copyt['index'][rank_sort][:2])
    if temp == []:
        under_20.append((title, [ind[np.argmin(rank_copyt[title])]]))
    else:
        under_20.append((title, temp))

In [ ]:
k_food = []
place = []
k_slang = []
k_fashion_beauty = []
k_pop = []
history_event = []
k_culture = []
for drama, category in under_20:
    for i in category:
        if i == 'rank_food':
            k_food.append(drama)
        elif i == 'rank_place':
            place.append(drama)
        elif i == 'rank_slang':
            k_slang.append(drama)
        elif i == 'rank_fashion_beauty':
            k_fashion_beauty.append(drama)
        elif i == 'rank_pop':
            k_pop.append(drama)
        elif i == 'rank_history_event':
            history_event.append(drama)
        else:
            k_culture.append(drama)

In [ ]:
k_dict = {'k_food': [], 'place': [], 'k_slang': [], 'k_fashion_beauty': [
], 'k_pop': [], 'history_event': [], 'k_culture': []}
for drama, category in under_20:
    for i in category:
        if i == 'rank_food':
            k_dict['k_food'].append(drama)
        elif i == 'rank_place':
            k_dict['place'].append(drama)
        elif i == 'rank_slang':
            k_dict['k_slang'].append(drama)
        elif i == 'rank_fashion_beauty':
            k_dict['k_fashion_beauty'].append(drama)
        elif i == 'rank_pop':
            k_dict['k_pop'].append(drama)
        elif i == 'rank_history_event':
            k_dict['history_event'].append(drama)
        else:
            k_dict['k_culture'].append(drama)

# Sentiment

In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

In [ ]:
word_to_index = imdb.get_word_index()
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value+3] = key

In [ ]:
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index] = token

## GRU

In [ ]:
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,100,input_length=max_len),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('BiLSTM_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(
     loss='binary_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [ ]:
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2, workers=-1)

In [ ]:
loaded_model = load_model('/Users/suchan/study/Final_Project/0506_GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
def sentiment_predict(new_sentence):
    # 알파벳과 숫자를 제외하고 모두 제거 및 알파벳 소문자화
    new_sentence = re.sub('[^0-9a-zA-Z ]', '', new_sentence).lower()
    encoded = []

    # 띄어쓰기 단위 토큰화 후 정수 인코딩
    for word in new_sentence.split():
        try:
            # 단어 집합의 크기를 10,000으로 제한.
            if word_to_index[word] <= 10000:
                encoded.append(word_to_index[word]+3)
            else:
                # 10,000 이상의 숫자는 <unk> 토큰으로 변환.
                encoded.append(2)
        # 단어 집합에 없는 단어는 <unk> 토큰으로 변환.
        except:
            encoded.append(2)

    pad_sequence = pad_sequences([encoded], maxlen=max_len)
    score = float(loaded_model.predict(pad_sequence))  # 예측

    if(score > 0.6):
        return 0
    else:
        return 1

In [ ]:
pos_neg = []
for i in tqdm(range(len(eng['reviews']))):
    sentiment = sentiment_predict(eng['reviews'][i])
    pos_neg.append(sentiment)

In [ ]:
eng['pos_neg'] = pos_neg

In [ ]:
eng.to_csv('0506_ENG_Korean_plus_Sentiment.csv',index=False)

# Keyword Extraction

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/0506_ENG_Korean_plus_Sentiment.csv")

In [ ]:
drama_theme = [('악의마음을읽는자들', ['rank_history_event', 'rank_place']),
    ('라이프온마스', ['rank_place', 'rank_food']),
    ('옷소매붉은끝동', ['rank_history_event']),
    ('이구역의미친X', ['rank_food', 'rank_fashion_beauty']),
    ('어느날우리집현관으로멸망이들어왔다', ['rank_pop', 'rank_food']),
    ('스물다섯스물하나', ['rank_pop']),
    ('무브투헤븐:나는유품정리사입니다', ['rank_culture']),
    ('호텔델루나', ['rank_pop', 'rank_fashion_beauty']),
    ('하늘에서내리는일억개의별', ['rank_culture']),
    ('기름진멜로', ['rank_food', 'rank_slang']),
    ('악의꽃', ['rank_culture']),
    ('서른이지만열일곱입니다', ['rank_pop']),
    ('킹덤', ['rank_history_event', 'rank_place']),
    ('작은신의아이들', ['rank_culture']),
    ('검색어를입력하세요WWW', ['rank_slang', 'rank_pop']),
    ('술꾼도시여자들', ['rank_food', 'rank_place']),
    ('한번다녀왔습니다', ['rank_food', 'rank_slang']),
    ('서른아홉', ['rank_food', 'rank_slang']),
    ('오월의청춘', ['rank_history_event', 'rank_place']),
    ('트랩', ['rank_slang']),
    ('왓쳐', ['rank_food', 'rank_pop']),
    ('아는와이프', ['rank_slang', 'rank_pop']),
    ('자백', ['rank_culture']),
    ('60일지정생존자', ['rank_place']),
    ('시맨틱에러', ['rank_culture']),
    ('녹두꽃', ['rank_history_event']),
    ('백일의낭군님', ['rank_history_event', 'rank_pop']),
    ('괴물', ['rank_food', 'rank_history_event']),
    ('태종이방원', ['rank_culture', 'rank_history_event']),
    ('사이코지만괜찮아', ['rank_fashion_beauty', 'rank_food']),
    ('런온', ['rank_food']),
    ('365:운명을거스르는1년', ['rank_slang']),
    ('시를잊은그대에게', ['rank_culture']),
    ('인간수업', ['rank_place']),
    ('으라차차와이키키', ['rank_culture']),
    ('우수무당가두심', ['rank_place', 'rank_history_event']),
    ('트레이서', ['rank_culture']),
    ('사내맞선', ['rank_culture', 'rank_food']),
    ('구경이', ['rank_pop', 'rank_food']),
    ('어쩌다발견한하루', ['rank_history_event', 'rank_culture']),
    ('김비서가왜그럴까', ['rank_fashion_beauty', 'rank_food']),
    ('나의아저씨', ['rank_pop']),
    ('카이로스', ['rank_culture', 'rank_slang']),
    ('18어게인', ['rank_pop']),
    ('D.P.', ['rank_place']),
    ('로스쿨', ['rank_pop']),
    ('대박부동산', ['rank_slang', 'rank_fashion_beauty']),
    ('사랑의불시착', ['rank_place']),
    ('스토브리그', ['rank_place', 'rank_fashion_beauty']),
    ('단하나의사랑', ['rank_pop', 'rank_slang']),
    ('안녕나야', ['rank_slang', 'rank_fashion_beauty']),
    ('그림자미녀', ['rank_fashion_beauty', 'rank_culture']),
    ('도시남녀의사랑법', ['rank_place']),
    ('본대로말하라', ['rank_culture']),
    ('내뒤에테리우스', ['rank_slang']),
    ('월간집', ['rank_food']),
    ('비밀의숲', ['rank_food', 'rank_place']),
    ('나빌레라', ['rank_fashion_beauty']),
    ('슬기로운의사생활', ['rank_food', 'rank_pop']),
    ('스카이캐슬', ['rank_place']),
    ('하이에나', ['rank_food', 'rank_fashion_beauty']),
    ('라이브', ['rank_place']),
    ('갯마을차차차', ['rank_food']),
    ('소년심판', ['rank_place']),
    ('그녀의사생활', ['rank_fashion_beauty']),
    ('손theguest', ['rank_food']),
    ('세빛남고학생회', ['rank_fashion_beauty', 'rank_place']),
    ('오징어게임', ['rank_culture']),
    ('원더우먼', ['rank_slang', 'rank_fashion_beauty']),
    ('우아한친구들', ['rank_pop']),
    ('낭만닥터김사부', ['rank_pop']),
    ('마우스', ['rank_food', 'rank_fashion_beauty']),
    ('지옥', ['rank_place']),
    ('왕이된남자', ['rank_history_event', 'rank_culture']),
    ('라켓소년단', ['rank_culture']),
    ('유미의세포들', ['rank_fashion_beauty']),
    ('부부의세계', ['rank_slang']),
    ('로맨스는별책부록', ['rank_slang', 'rank_fashion_beauty']),
    ('미스티', ['rank_fashion_beauty']),
    ('미치지않고서야', ['rank_culture']),
    ('마녀식당으로오세요', ['rank_place', 'rank_fashion_beauty']),
    ('미스터션샤인', ['rank_history_event']),
    ('모범택시', ['rank_history_event']),
    ('사의찬미', ['rank_fashion_beauty', 'rank_pop']),
    ('멜로가체질', ['rank_pop']),
    ('이리와안아줘', ['rank_fashion_beauty', 'rank_slang']),
    ('보좌관:세상을움직이는사람들', ['rank_pop']),
    ('뷰티인사이드', ['rank_pop', 'rank_slang']),
    ('우리들의블루스', ['rank_place', 'rank_history_event']),
    ('해피니스', ['rank_history_event']),
    ('의사요한', ['rank_pop']),
    ('동백꽃필무렵', ['rank_culture'])]

In [ ]:
samples = df.copy()
samples.columns
samples.rename(columns={'k_food':'rank_food', 'place':'rank_place', 'k_slang' : 'rank_slang',
       'k_fashion_beauty':'rank_fashion_beauty', 'k_pop' : 'rank_pop',
       'history_event' : 'rank_history_event', 'k_culture' : 'rank_culture'},inplace=True)
samples[:3]

In [ ]:
pos_df = samples[samples.pos_neg == 0]

In [ ]:
import ast
drama_food_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    food_keywords = []
    for i in keyword_list:
        if i in k_food:
            food_keywords.append(i)

    wordCount = {}

    for word in food_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_food_keywords.append((drama, sort_words))
drama_food_keywords

In [ ]:
drama_place_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    place_keywords = []
    for i in keyword_list:
        if i in place:
            place_keywords.append(i)

    wordCount = {}

    for word in place_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_place_keywords.append((drama, sort_words))

In [ ]:
drama_slang_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    slang_keywords = []
    for i in keyword_list:
        if i in k_slang:
            slang_keywords.append(i)

    wordCount = {}

    for word in slang_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_slang_keywords.append((drama, sort_words))

In [ ]:
drama_fashion_beauty_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    fashion_beauty_keywords = []
    for i in keyword_list:
        if i in k_fashion_beauty:
            fashion_beauty_keywords.append(i)

    wordCount = {}

    for word in fashion_beauty_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_fashion_beauty_keywords.append((drama, sort_words))

In [ ]:
drama_pop_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    pop_keywords = []
    for i in keyword_list:
        if i in k_pop:
            pop_keywords.append(i)

    wordCount = {}

    for word in pop_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_pop_keywords.append((drama, sort_words))

In [ ]:
drama_history_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    history_keywords = []
    for i in keyword_list:
        if i in k_food:
            history_keywords.append(i)

    wordCount = {}

    for word in history_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_history_keywords.append((drama, sort_words))

In [ ]:
drama_culture_keywords = []
for drama in pos_df['title'].unique():
    keyword_list = []
    for li in pos_df[(pos_df.title == drama)]['K_words_list']:
        str_list = ast.literal_eval(li)
        for keyword in str_list:
            keyword_list.append(keyword)
    culture_keywords = []
    for i in keyword_list:
        if i in k_food:
            culture_keywords.append(i)

    wordCount = {}

    for word in culture_keywords:

        # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴

        wordCount[word] = wordCount.get(word, 0) + 1

        keys = sorted(wordCount.keys())

    sort_words = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
    drama_culture_keywords.append((drama, sort_words))

# Review Summarize

In [ ]:
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
torch_device = 'cpu'

In [ ]:
def bart_summarize(text, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):

    text = text.replace('\n', '')
    text_input_ids = tokenizer.batch_encode_plus(
        [text], return_tensors='pt', max_length=1024, truncation=True)['input_ids'].to(torch_device)
    summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(
        length_penalty), max_length=int(max_length), min_length=int(min_length), no_repeat_ngram_size=int(no_repeat_ngram_size))
    summary_txt = tokenizer.decode(
        summary_ids.squeeze(), skip_special_tokens=True)
    return summary_txt

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv("/Users/suchan/study/Final_Project/0506_ENG_Korean_plus_Sentiment.csv")

In [ ]:
df.drop(df[df.pos_neg == 1].index, axis=0, inplace=True)

In [ ]:
reviews = df.reviews.tolist()

In [ ]:
review_summarize = []
for review in tqdm(reviews):
    review_summarize.append(bart_summarize(review, 4, 4.0, 142, 5, 3))

In [ ]:
df_review_summarize = pd.DataFrame(review_summarize)

In [ ]:
df_review_summarize.to_csv("review_summarize.csv",index=False)